In [186]:
import sqlite3
conn = sqlite3.connect('../data/database.db')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix


In [187]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [188]:
df = pd.read_sql_query("SELECT * FROM warehouse", conn)


In [189]:
df.head()

,id,date,week,favorite,underdog,point_spread,over_under,local,winner,pointsW,pointsL,loser
0,1,2002-09-05,1,San Francisco 49ers,New York Giants,-3.5,38.5,New York Giants,San Francisco 49ers,16,13,New York Giants
1,2,2002-09-08,1,Chicago Bears,Minnesota Vikings,-4.5,41.0,Chicago Bears,Chicago Bears,27,23,Minnesota Vikings
2,3,2002-09-08,1,Baltimore Ravens,Carolina Panthers,-2.0,33.5,Carolina Panthers,Carolina Panthers,10,7,Baltimore Ravens
3,4,2002-09-08,1,New York Jets,Buffalo Bills,-3.0,40.5,Buffalo Bills,New York Jets,37,31,Buffalo Bills
4,5,2002-09-08,1,Miami Dolphins,Detroit Lions,-8.5,36.0,Miami Dolphins,Miami Dolphins,49,21,Detroit Lions


In [190]:
nnframe = pd.DataFrame(data={
    'right_guess':  (df['favorite'] == df['winner']).astype(int),
    'point_spread': df['point_spread'],
    'over_under': df['over_under'],
    'favorite_is_local': (df['favorite'] == df['local']).astype(int),
})
nnframe.head()

,right_guess,point_spread,over_under,favorite_is_local
0,1,-3.5,38.5,0
1,1,-4.5,41.0,1
2,0,-2.0,33.5,0
3,1,-3.0,40.5,0
4,1,-8.5,36.0,1


## separate training from testing data

In [191]:
X = nnframe.drop('right_guess', axis=1)
Y = nnframe['right_guess']

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [193]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [194]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train

array([[-2.42565701, -0.33626561,  0.71228206],
       [ 0.98220095,  0.19599066,  0.71228206],
       [-0.05497322, -1.61368068,  0.71228206],
       ...,
       [ 0.24136226,  0.72824694, -1.4039382 ],
       [-1.38848285,  0.30244192,  0.71228206],
       [-0.64764416,  0.62179568,  0.71228206]])

## Training

In [198]:
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10),max_iter=600)


In [199]:
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=600, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

## Predict

In [200]:
predictions = mlp.predict(X_test)

In [203]:
print(confusion_matrix(y_test,predictions))

[[  1 335]
 [  4 686]]


In [204]:
print(classification_report(y_test,predictions))


             precision    recall  f1-score   support

          0       0.20      0.00      0.01       336
          1       0.67      0.99      0.80       690

avg / total       0.52      0.67      0.54      1026



precision = % of times it predicts a win and it is a win.

recall = % of times it predicts a win over the overall total of wins

In [185]:
confusion_matrix([1, 0],[1,0])

array([[1, 0],
       [0, 1]])